# 판교 AI Challenge
> 참치김치찌개팀<br>
> 팀장 손찬영, 팀원 김민정 김하림 이두현 차현수
* 과제명 : [아동 및 교통약자 보호를 위한 어린이 도로보행 위험행동 분류 과제]
* 과제 링크 : https://www.aiconnect.kr/main/competition/privateDetail/200

----------------------------------------------------------------

## Package

In [1]:
import glob
import math
import os
import random
import sys
import time
import timeit
import warnings

import easydict

warnings.filterwarnings("ignore")

# Others
from datetime import datetime

import numpy as np
import pandas as pd

# Customized Source Python Files
import source.dataset as dataset

# Pytorch
import torch
import wandb
from sklearn.metrics import f1_score
from source.model import C3D_model, R2Plus1D_model, R3D_model
from source.model.utils.vit import TimeSformer
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
%%wandb
#%env WANDB_SILENT=True
%env "WANDB_NOTEBOOK_NAME" "main"

env: "WANDB_NOTEBOOK_NAME"="main"


----------------------------------------------------------------

## Parameter Settings

In [3]:
args = easydict.EasyDict(
    {
        "experiment": "exp1 TimeSformer",  # 매번 바꿔준다.
        "wandb": True,
        "randomseed": False,
        "dataset": "kids",  # Options: hmdb51 or ucf101 or `kids`
        "dataset_root_dir": "./dataset",
        "project_dir": os.getcwd(),
        "model_dir": "./pretrained/TimeSformer_divST_8x32_224_K400.pyth",
        "snapshot": 20,  # Store a model every snapshot epochs
        "clip_len": 16,
        "num_workers": 4,
        "save_epoch": 5,  # SAVE, View
        "model": "TimeSformer",  # Options: C3D or R2Plus1D or R3D
        "attention_type": "divided_space_time",
        "num_frames": 19,
        "img_size": 224,
        "count": 20,  # how many repeat for wandb
    }
)
NAME_ELEMENTS = [args.model, time.strftime("%m%d_%H%M", time.localtime(time.time()))]
MODEL_NAME = "_".join(NAME_ELEMENTS)

----------------------------------------------------------------

## W&B & Randomseed

### 🚀 Setup

Start out by installing the experiment tracking library and setting up your free W&B account:<br>
`.login()` so you can log metrics to your projects<br>
If you've never used Weights & Biases before,
the call to `login` will give you a link to sign up for an account.
W&B is free to use for personal and academic projects!<br>
### 👈 Pick a `method`
The first thing we need to define is the `method`
for choosing new parameter values.

We provide the following search `methods`:
*   **`grid` Search** – Iterate over every combination of hyperparameter values.
Very effective, but can be computationally costly.
*   **`random` Search** – Select each new combination at random according to provided `distribution`s. Surprisingly effective!
*   **`bayesian` Search** – Create a probabilistic model of metric score as a function of the hyperparameters, and choose parameters with high probability of improving the metric. Works well for small numbers of continuous parameters but scales poorly.

We'll stick with `random`.

In [4]:
if args.wandb:
    wandb.login()
    sweep_config = {
        "name": args.experiment,
        "method": "bayes",  # grid, bayesian, random
        "metric": {
            "name": "Weighted F1 Scrore",
            "goal": "maximize",
            # 'target': 'goal value for the metric you're optimizing, for example : 0.01'
        },
        "parameters": {
            "epochs": {"values": [10, 15, 20]},
            "learning_rate": {"distribution": "uniform", "min": 1e-4, "max": 5e-2,},
            "optimizer": {"values": ["adam", "sgd", "adamw", "adadelta", "nadam"]},
            "loss_function": {"values": ["focal", "cross_entropy", "label_smooth"]},
            "schedular": {
                "values": ["step", "onecycle", "cosineannealingwarmrestarts", "swa"]
            },
            "batch_size": {"values": [10, 15]},
        },
    }
    sweep_id = wandb.sweep(sweep_config, project="tunakimchi")
    os.environ["WANDB_NOTEBOOK_NAME"] = "main.ipynb"

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: stephencha (use `wandb login --relogin` to force relogin)


Create sweep with ID: x0zq7vyf
Sweep URL: https://wandb.ai/stephencha/tunakimchi/sweeps/x0zq7vyf


In [5]:
if args.randomseed:
    torch.manual_seed(args.randomseed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(args.randomseed)
    random.seed(args.randomseed)

----------------------------------------------------------------

## Model

In [6]:
# Use GPU if available else revert to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device being used:", device)

if args.dataset == "hmdb51":
    num_classes = 51
elif args.dataset == "ucf101":
    num_classes = 101
elif args.dataset == "kids":
    num_classes = 9
else:
    print("We only implemented hmdb and ucf datasets.")
    raise NotImplementedError

saveName = args.model + "-" + args.dataset

Device being used: cuda


In [7]:
# build run dir
runs = sorted(glob.glob(os.path.join(args.project_dir, "run", "run_*")))
run_id = int(runs[-1].split("_")[-1]) + 1 if runs else 0

SAVE_DIR = os.path.join(args.project_dir, "run", "run_" + str(run_id))
model_save_dir = os.path.join(SAVE_DIR, "models")
os.makedirs(model_save_dir, exist_ok=True)


if args.model == "C3D":
    model = C3D_model.C3D(model_dir=model_dir, num_classes=num_classes, pretrained=True)
    train_params = [
        {"params": C3D_model.get_1x_lr_params(model), "lr": lr},
        {"params": C3D_model.get_10x_lr_params(model), "lr": lr * 10},
    ]

elif args.model == "R2Plus1D":
    model = R2Plus1D_model.R2Plus1DClassifier(
        num_classes=num_classes, layer_sizes=(2, 2, 2, 2)
    )
    train_params = [
        {"params": R2Plus1D_model.get_1x_lr_params(model), "lr": lr},
        {"params": R2Plus1D_model.get_10x_lr_params(model), "lr": lr * 10},
    ]

elif args.model == "R3D":
    model = R3D_model.R3DClassifier(num_classes=num_classes, layer_sizes=(2, 2, 2, 2))
    train_params = model.parameters()

elif args.model == "TimeSformer":
    model = TimeSformer(
        img_size=args.img_size,
        num_classes=num_classes,
        num_frames=args.num_frames,
        attention_type=args.attention_type,
        pretrained_model=args.model_dir,
    )
    train_params = model.parameters()

else:
    raise NotImplementedError

In [8]:
print("Training {} from scratch...".format(args.model))
print("Total params: %.2fM" % (sum(p.numel() for p in model.parameters()) / 1000000.0))
# model = nn.DataParallel(model, device_ids=[0, 1])
model.to(device)

Training TimeSformer from scratch...
Total params: 121.27M


TimeSformer(
  (model): VisionTransformer(
    (dropout): Dropout(p=0.0, inplace=False)
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (time_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
          (attn_drop): Dropout(p=0.0, inplace=False)
        )
        (temporal_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (temporal_attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
   

----------------------------------------------------------------

## Dataset Preprocessing and Loading

* 베이스라인 학습을 위한 데이터 경로 설정

원활한 베이스라인 학습을 위하여 `dataset` 디렉토리 안에 다음과 같이 경로를 설정하여 데이터를 저장해야한다.<br>
train 데이터셋은 카테고리별로 별도의 디렉토리에 비디오를 분리하여 저장해야 하는데, arrangement.ipynb를 이용하면 바로 분리하여 저장시켜준다.

```
dataset
├── label
│   └── kids_labels
├── train
│   ├── driveway_walk
│   │   ├── train_0003.mp4
│   │   └── ...
│   ├── fall_down
│   │   ├── train_0002.mp4
│   │   └── ...
│   └── fighting
│   │   ├── train_0056.mp4
│   │   └── ...
│   ...
└── test (공개가 안되어있다)
    ├── test_0000.mp4
    ├── test_0000.mp4
    ├── test_0000.mp4
    ├── ...
```

학습 및 추론 전처리 과정에서 각각 train_processed, test_processed 디렉토리가 다음과 같이 생성된다.<br>
이는 dataset.py를 실행시키면 되나, 굳이 할 필요는 없다.<br>
전처리에서는 비디오에서 16프레임을 샘플링한 이미지 데이터를 비디오 파일명 디렉토리에 별도로 저장하는 과정이 수행된다.
```
dataset
├── label
│   └── kids_labels
├── train
│   ├── driveway_walk
│   │   ├── train_0003.mp4
│   │   └── ...
│   ├── fall_down
│   │   ├── train_0002.mp4
│   │   └── ...
│   └── fighting
│   │   ├── train_0056.mp4
│   │   └── ...
│   ...
│
├── train_processed
│   ├── train
│   │   ├── driveway_walk
│   │   │   ├── train_0003
│   │   │   │   ├── 00000.jpg
│   │   │   │   ├── 00001.jpg
│   │   │   └── ...
│   │   └── ...
│   └── val
│   │   ├── driveway_walk
│   │   │   ├── train_0004
│   │   │   │   ├── 00000.jpg
│   │   │   │   ├── 00001.jpg
│   │   │   │   └── ...
│   │   │   └── ...
│   │   └── ...
│
├── test
│   ├── test_0000.mp4
│   ├── test_0000.mp4
│   ├── test_0000.mp4
│   └── ...
│
└── test_processed
    ├── test_0000
    │   ├── 00000.jpg
    │   ├── 00001.jpg
    │   └── ...
    ├── test_0001
    │   ├── 00000.jpg
    │   ├── 00001.jpg
    │   └── ...
    ├── ...
```

In [9]:
print("Training model on {} dataset...".format(dataset))
train_dataset = dataset.VideoDataset(
    root_dir=args.dataset_root_dir,
    dataset=args.dataset,
    split="train",
    clip_len=args.clip_len,
)
val_dataset = dataset.VideoDataset(
    root_dir=args.dataset_root_dir,
    dataset=args.dataset,
    split="val",
    clip_len=args.clip_len,
)


def build_dataset(batch_size):
    train_dataloader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=args.num_workers
    )
    val_dataloader = DataLoader(
        val_dataset, batch_size=batch_size, num_workers=args.num_workers
    )

    trainval_loaders = {"train": train_dataloader, "val": val_dataloader}
    trainval_sizes = {x: len(trainval_loaders[x].dataset) for x in ["train", "val"]}

    return trainval_loaders, trainval_sizes

Training model on <module 'source.dataset' from '/home/stephencha/Hub/soo/source/dataset.py'> dataset...
Number of train videos: 2663
Number of val videos: 670


----------------------------------------------------------------

## Loss Function and Optimizer

In [10]:
import timm.optim.nadam as nadam
import torchcontrib
from source.focalloss import FocalLoss
from source.label_smooth import LabelSmoothSoftmaxCEV2
from torch.optim.swa_utils import SWALR
from torchcontrib.optim import SWA

In [11]:
def build_loss_function(lf):
    if lf == "focal":
        lf = FocalLoss()
    elif lf == "cross_entropy":
        lf = nn.CrossEntropyLoss()
    elif lf == "label_smooth":
        lf = LabelSmoothSoftmaxCEV2()
    return lf

In [12]:
def build_optimizer(model, opt, lr):
    if args.model == "C3D":
        param = [
            {"params": C3D_model.get_1x_lr_params(model), "lr": lr},
            {"params": C3D_model.get_10x_lr_params(model), "lr": lr * 10},
        ]
    elif args.model == "R2Plus1D":
        param = [
            {"params": R2Plus1D_model.get_1x_lr_params(model), "lr": lr},
            {"params": R2Plus1D_model.get_10x_lr_params(model), "lr": lr * 10},
        ]
    elif args.model == "R3D":
        param = model.parameters()
    elif args.model == "TimeSformer":
        param = model.parameters()
    else:
        raise NotImplementedError

    if opt == "sgd":
        optimizer = optim.SGD(param, lr=lr, momentum=0.9, weight_decay=5e-4)
    elif opt == "adam":
        optimizer = optim.Adam(param, lr=lr, amsgrad=True)
    elif opt == "adamw":
        optimizer = optim.AdamW(param, lr=lr)
    elif opt == "adadelta":
        optimizer = optim.Adadelta(param, lr=lr)
    elif opt == "nadam":
        optimizer = nadam.Nadam(param, lr=lr)
    return optimizer

In [13]:
def build_schedular(optimizer, sche, epochs, length):
    if sche == "step":
        schedular = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    elif sche == "onecycle":
        schedular = optim.lr_scheduler.OneCycleLR(
            optimizer,
            pct_start=0.1,
            div_factor=1e5,
            max_lr=0.0001,
            epochs=epochs,
            steps_per_epoch=length,
        )
    elif sche == "cosineannealingwarmrestarts":
        schedular = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=10, T_mult=2, eta_min=1e-5, last_epoch=-1
        )
    elif sche == "swa":
        schedular = SWALR(optimizer, swa_lr=0.01)
    return schedular

----------------------------------------------------------------

## Train
* epoch마다 학습과 검증을 실시한다.
* `./run/run_*` 디렉토리에서 저장된 가중치 파일을 확인한다.

In [14]:
def train():
    with wandb.init(config=args, name=MODEL_NAME) as run:  # reinit=True
        config = wandb.config

        trainval_loaders, trainval_sizes = build_dataset(config.batch_size)

        # standard crossentropy loss for classification
        criterion = build_loss_function(config.loss_function)
        optimizer = build_optimizer(
            model, opt=config.optimizer, lr=config.learning_rate
        )

        # the scheduler divides the lr by 10 every 10 epochs
        if config.schedular == "swa":
            optimizer = torchcontrib.optim.SWA(optimizer)

        scheduler = build_schedular(
            optimizer,
            sche=config.schedular,
            epochs=config.epochs,
            length=len(trainval_loaders["train"]),
        )

        best_score = np.Inf
        for epoch in range(config.epochs):
            # each epoch has a training and validation step
            for phase in ["train", "val"]:
                start_time = timeit.default_timer()

                # reset the running loss and corrects
                running_loss = 0.0
                running_corrects = 0.0

                # set model to train() or eval() mode depending on whether it is trained
                # or being validated. Primarily affects layers such as BatchNorm or Dropout.
                if phase == "train":
                    # scheduler.step() is to be called once every epoch during training
                    scheduler.step()
                    model.train()
                else:
                    model.eval()

                epoch_labels = []
                epoch_preds = []

                for inputs, labels in tqdm(trainval_loaders[phase]):
                    # move inputs and labels to the device the training is taking place on
                    inputs = Variable(inputs, requires_grad=True).to(device)
                    labels = Variable(labels).to(device)
                    optimizer.zero_grad()

                    if phase == "train":
                        outputs = model(inputs)
                    else:
                        with torch.no_grad():
                            outputs = model(inputs)

                    probs = nn.Softmax(dim=1)(outputs)
                    preds = torch.max(probs, 1)[1]
                    loss = criterion(outputs, labels)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                    epoch_labels.extend(labels.tolist())
                    epoch_preds.extend(preds.tolist())

                epoch_loss = running_loss / trainval_sizes[phase]
                epoch_acc = running_corrects.double() / trainval_sizes[phase]

                epoch_score = f1_score(epoch_preds, epoch_labels, average="weighted")
                print(f"{phase} | EPOCH {epoch} Weighted F1 SCORE: {epoch_score}")

                print(
                    "[{}] Epoch: {}/{} Loss: {} Acc: {}".format(
                        phase, epoch + 1, config.epochs, epoch_loss, epoch_acc
                    )
                )
                stop_time = timeit.default_timer()
                print("Execution time: " + str(stop_time - start_time) + "\n")

                if epoch_score < best_score:
                    print(
                        f"Validation Weighted F1 Score decreased ({best_score:.6f} --> {epoch_score:.6f}).  Saving model ..."
                    )
                    path_dir = [SAVE_DIR, "{:.6f}.pt".format(epoch_score)]
                    torch.save(model.state_dict(), os.path.join(*path_dir))
                    best_score = epoch_score

                if args.wandb and phase == "val":
                    wandb.log(
                        {"Weighted F1 Scrore": epoch_score,}
                    )

            if epoch % args.save_epoch == (args.save_epoch - 1):
                model_path = os.path.join(
                    SAVE_DIR, "models", saveName + "_epoch-" + str(epoch) + ".pth.tar"
                )
                torch.save(
                    {
                        "epoch": epoch + 1,
                        "state_dict": model.state_dict(),
                        "opt_dict": optimizer.state_dict(),
                    },
                    model_path,
                )
                print("Save model at {}\n".format(model_path))

In [ ]:
if args.wandb:
    wandb.agent(sweep_id, train, count=args.count)
else:
    train()

wandb: Agent Starting Run: 702r9l3a with config:
wandb: 	batch_size: 15
wandb: 	epochs: 10
wandb: 	learning_rate: 0.04401988009096347
wandb: 	loss_function: cross_entropy
wandb: 	optimizer: adam
wandb: 	schedular: cosineannealingwarmrestarts


100%|██████████| 178/178 [17:42<00:00,  5.97s/it]


train | EPOCH 0 Weighted F1 SCORE: 0.33650511296900487
[train] Epoch: 1/10 Loss: 4.0444980202605745 Acc: 0.2827638002253098
Execution time: 1062.3134663130004

Validation Weighted F1 Score decreased (inf --> 0.336505).  Saving model ...


  0%|          | 0/178 [00:00<?, ?it/s]

val | EPOCH 0 Weighted F1 SCORE: 0.5044642857142857
[val] Epoch: 1/10 Loss: 1.9998585510609754 Acc: 0.3373134328358209
Execution time: 7.453054837000309



 20%|██        | 36/178 [03:35<14:09,  5.98s/it]

In [ ]:
if args.wandb:
    run.finish()
print("Done!")

----------------------------------------------------------------

## Inference
* 루트 디렉토리에 생성된 `submit.csv` 파일을 확인하고, 제출합니다.

In [ ]:
cls_li = [
    "driveway_walk",
    "fall_down",
    "fighting",
    "jay_walk",
    "normal",
    "putup_umbrella",
    "ride_cycle",
    "ride_kick",
    "ride_moto",
]

DATA_DIR = args.dataset_root_dir  # os.path.join(PROJECT_DIR, '')
batch_size = 4

In [ ]:
checkpoint = torch.load(
    model_path, map_location=lambda storage, loc: storage
)  # Load all tensors onto the CPU
print(f"Initializing weights from: {model_path.split('/')[-1]}...")
model.load_state_dict(checkpoint["state_dict"])
print("Total params: %.2fM" % (sum(p.numel() for p in model.parameters()) / 1000000.0))

print("Model Inference on {} dataset...".format(dataset))

if os.path.isdir(os.path.join(DATA_DIR, "test_processed")):
    preprocess = False
else:
    preprocess = True

test_dataset = dataset.TestDataset(dataset=dataset, clip_len=16, preprocess=preprocess)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=4)

model.eval()
start_time = timeit.default_timer()

pred_li = []
for inputs in tqdm(test_dataloader):
    inputs = inputs.to(device)

    with torch.no_grad():
        outputs = model(inputs)

    probs = nn.Softmax(dim=1)(outputs)
    preds = torch.max(probs, 1)[1]
    pred_li.extend(preds.tolist())

stop_time = timeit.default_timer()
print("Execution time: " + str(stop_time - start_time) + "\n")

sample_submission = pd.read_csv("sample_submission.csv")
sample_submission["class"] = [cls_li[int(pred)] for pred in pred_li]
sample_submission.to_csv("submit.csv", index=False)

----------------------------------------------------------------